# Many Variables  GHZ

In [1]:
print('Started!')
from system import *
s1 = system('2-',MMA=True,ManyVariables = False)
#s1 = load_object('x-o-o-')

Started!
Initializing system 2-  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...

System  2-  initialized in 4.5 seconds.


# Effective Hamiltonian Matrix obtain 

In [2]:
#phi_val = np.pi/180 * 30
n = 0

#total_indices = s1.variable_index

var('De1', domain='positive')
var('Deg1', domain='positive' , latex_name =r'{{\Delta e}}_{{  1}}^\gamma')
var('De2', domain='positive')
var('Deg2', domain='positive' , latex_name =r'{{\Delta e}}_{{  2}}^\gamma')
var('De3', domain='positive')
var('Deg3', domain='positive' , latex_name =r'{{\Delta e}}_{{  3}}^\gamma')    

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g','phi',domain='positive')
var('DEg', domain='positive' , latex_name =r'{{\Delta E}}^\gamma')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')
var('r_f',domain='positive',latex_name =r'r_f')
var('r_g',domain='positive',latex_name =r'r_g')
var('R_f',domain='positive')#ratio  (g_f/g)^2
eff_hamiltonian_tayl = sg.copy( s1.eff_hamiltonian.parent().zero())
for diag in s1.pos_gs:
    print(f'Calculating {diag} out of {s1.pos_gs}...')
    min_expr_len = 10**10
    for term in range(1):
        b = s1.eff_hamiltonian[diag,diag]
        b = b.operands()[1].operands()[term]
        if len(str(b))<10: #we isolated Omega^2 instead of one of the conjugate pairs
            b = s1.eff_hamiltonian[diag,diag].operands()[0].operands()[term]
        
        b = b._mathematica_().ReplaceAll(f'DE->DEg*gamma*C^({n})')._sage_()
        #b = b.subs(DE = DEg*gamma*C^n)
        
        b = b._mathematica_().ReplaceAll(f'De->Deg*gamma*C^({n})')._sage_()
        #b = b.subs(De1 = Deg1*gamma*C^n) #sqrt C scaling!?
        
        #b = b._mathematica_().ReplaceAll(f'De2->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De2 = Deg2*gamma*C^n) #MAKE ZERO
        
        #b = b._mathematica_().ReplaceAll(f'De3->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De3 = Deg3*gamma*C^n) #MAKE ZERO  
        
        #b = b.subs(phi = phi_val)
        
        #b = MMA_simplify(b)
        
        #b = b._mathematica_().ReplaceAll('g_f->g*(R_f)^(1/2)')._sage_()
        b = b.subs(g_f=g*(R_f)^(1/2))

        b = b._mathematica_().ReplaceAll('gamma_f->gamma*r_f')._sage_()
        #b = b.subs(gamma_f=gamma*r_f)

        b = b._mathematica_().ReplaceAll('gamma_g->gamma*r_g')._sage_()
        #b = b.subs(gamma_g=gamma*r_g)
        
        #b = b._mathematica_().ReplaceAll("g-> (C*gamma*kappa_c)^(1/2)")._sage_()
        b = b.subs(g = (C*gamma*kappa_c)^(1/2)) 
        
        b = b._mathematica_().ReplaceAll('v->(c*kappa_b*kappa_c)^(1/2)')._sage_()
        #b = b.subs(v = (c*kappa_b*kappa_c)^(1/2) )
        
        

        #Substitutions
        #b = b.subs(c=np.pi,R_f=2.1,Deg=np.e,DEg=np.sqrt(5),r_g=np.sqrt(3),r_f=np.sqrt(2), phi=phi_val) 
        #b = b.subs(phi=phi_val)

        #b = b._mathematica_().Factor()._sage_()
        #b = b._mathematica_().Together()._sage_()


        b = SR(str(b).replace('Sqrt','sqrt'))
        if len(str(b))<min_expr_len:
            min_expr_len = len(str(b))
            h_eff = b
    
    #h_eff = h_eff.subs(c=C)
    h_eff = MMA_simplify(h_eff)
    '''
    h_eff = SR(str(h_eff).replace('Sqrt','sqrt'))
    var('x')
    h_eff = h_eff.subs(C=1/x)
    h_eff = MMA_simplify(h_eff)
    h_eff = h_eff._mathematica_().Series((x,0,1)).Normal()._sage_()
    h_eff = h_eff.subs(x=1/C)
    '''
    h_eff = SR(str(h_eff).replace('Sqrt','sqrt'))
    eff_hamiltonian_tayl[diag,diag] = h_eff  * Omega^2 / 2

print('\n Done! ')
#show(symround(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs],digits=3))


eff_hamiltonian_tayl_temp = eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs] 





lind_op_number = len(s1.lindblau_list)

eff_lind_taylor = []
eff_lind = []
eff_lind_coeff = []
for lind_op in range(lind_op_number):
    print(f'{lind_op} out of {lind_op_number-1}')
    eff_lind_taylor.append([])
    eff_lind.append([])
    L_matrix = s1.eff_lindblau_list[lind_op]
    L_nonzeros = []
    L_nonzeros_pos = []
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not str(symround(L_matrix[i,j]))==str(0):
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))
    eff_lind_coeff.append(s1.L_coeffs[lind_op])
    
    for which in range(len(L_nonzeros)):

        L_elem = L_nonzeros[which]
        b = L_elem
        
        b = b._mathematica_().ReplaceAll(f'DE->DEg*gamma*C^({n})')._sage_()
        #b = b.subs(DE = DEg*gamma*C^n)
        
        b = b._mathematica_().ReplaceAll(f'De->Deg*gamma*C^({n})')._sage_()
        #b = b.subs(De3 = Deg3*gamma*C^n) #MAKE ZERO  
        
        #b = b.subs(phi = phi_val)
        
        #b = MMA_simplify(b)
        
        #b = b._mathematica_().ReplaceAll('g_f->g*(R_f)^(1/2)')._sage_()
        b = b.subs(g_f=g*(R_f)^(1/2))

        b = b._mathematica_().ReplaceAll('gamma_f->gamma*r_f')._sage_()
        #b = b.subs(gamma_f=gamma*r_f)

        b = b._mathematica_().ReplaceAll('gamma_g->gamma*r_g')._sage_()
        #b = b.subs(gamma_g=gamma*r_g)
        
        #b = b._mathematica_().ReplaceAll("g-> (C*gamma*kappa_c)^(1/2)")._sage_()
        b = b.subs(g = (C*gamma*kappa_c)^(1/2)) 
        
        b = b._mathematica_().ReplaceAll('v->(c*kappa_b*kappa_c)^(1/2)')._sage_()
        
        b = b.subs(c=C)
        
        b = SR(str(b).replace('Sqrt','sqrt'))
        
        eff_lind[lind_op].append(b)
        
print('\n Done!')


eff_hamiltonian_tayl = eff_hamiltonian_tayl_temp

a = eff_hamiltonian_tayl[0,0]
#a = symround( MMA_simplify(a,full=False) ,digits = 14)
b = eff_hamiltonian_tayl[1,1]
#b = symround(MMA_simplify(b,full=False),digits = 14)
c = eff_hamiltonian_tayl[3,3]
#c = symround(MMA_simplify(c,full=False),digits = 14)
d = eff_hamiltonian_tayl[4,4]
#d = symround(MMA_simplify(d,full=False),digits = 14)
e = eff_hamiltonian_tayl[5,5]
#e = symround(MMA_simplify(e,full=False),digits = 14)
f = eff_hamiltonian_tayl[7,7]
#f = symround(MMA_simplify(f,full=False),digits = 14)

Calculating 0 out of [0, 1, 2, 3]...
Calculating 1 out of [0, 1, 2, 3]...
Calculating 2 out of [0, 1, 2, 3]...
Calculating 3 out of [0, 1, 2, 3]...

 Done! 
0 out of 5
1 out of 5
2 out of 5
3 out of 5
4 out of 5
5 out of 5

 Done!


IndexError: matrix index out of range

# See results

In [ ]:
m = 1

In [134]:
x = symround(a.subs(R_f=2.1,Deg=np.sqrt(2)*C^m,Deg2=np.e*C^m,DEg=np.sqrt(5)*C^m,r_g=np.sqrt(3),gamma=0.3,r_f=0.1*np.sqrt(2), phi=np.pi/10,gamma_f=1.45,gamma_g=3),digits=14)
x = x._mathematica_().Together().sage()
print('Numerator')
show(symround(x._mathematica_().Numerator()._sage_().expand(),digits=5))
print('\n\nDenominator')
show(symround(x._mathematica_().Denominator()._sage_().expand(),digits=5))

Numerator


(0.16461 - 0.30919*I)*Omega^2



Denominator


C + 1.3759 + 0.73252*I

In [111]:
#m = 3

lindop = 5
pos = 0

err = eff_lind[lindop][pos].subs(Omega=1,kappa_c=5.61,kappa_b=1.79, R_f=2.1,Deg=np.sqrt(2)*C^m,Deg2=np.e*C^m,DEg=np.sqrt(5)*C^m,r_g=np.sqrt(3),gamma=0.3,r_f=0.1*np.sqrt(2), phi=np.pi/10,gamma_f=1.45,gamma_g=3)

err = err._mathematica_().Together()._sage_() 
err = SR(str(( err)).replace('Sqrt','sqrt'))
#err = SR(str(MMA_simplify( err,full=False )).replace('Sqrt','sqrt')).simplify()
#show( symround( err ,digits=1) )
print(f'coeff:')
show(eff_lind_coeff[lindop])

coeff:


sqrt(gamma)

In [112]:
el = 0
q = (symround(err._mathematica_().Numerator()._sage_() ,tol=1e-30 ).expand().operands()[el])

print('Numerator')

show(symround(q._mathematica_().Numerator()._sage_(),digits=3).expand())

print(obtain_exponent(symround(q._mathematica_().Numerator()._sage_(),digits=3).expand()))

print('\n\nDenominator')
show(symround(q._mathematica_().Denominator()._sage_(),digits=3).expand())
print(obtain_exponent(symround(q._mathematica_().Denominator()._sage_(),digits=3).expand()))

print('\n\nextra Denominator')
show( symround(err._mathematica_().Denominator()._sage_().expand(),digits=3))

print(obtain_exponent(symround(err._mathematica_().Denominator()._sage_().expand(),digits=3)))

Numerator


-(132*I - 146)*C^3

3


Denominator


912*C^5 - (4064*I - 648)*C^4 - (2624*I + 2208)*C^3 - (412*I + 1152)*C^2 - (150. + 16.5*I)*C - 5.25

5


extra Denominator


1

0


In [113]:
show(symround(err._mathematica_().Numerator()._sage_().expand(),tol=1e-30))
#show(symround( err.numerator().expand()))

-(132*I - 146)*C^3/(912*C^5 - (4064*I - 648)*C^4 - (2624*I + 2208)*C^3 - (412*I + 1152)*C^2 - (150. + 16.*I)*C - 5.2) + (9.4 - 29.*I)*C^2/(912*C^5 - (4064*I - 648)*C^4 - (2624*I + 2208)*C^3 - (412*I + 1152)*C^2 - (150. + 16.*I)*C - 5.2)

In [114]:
#show(symround(err.denominator().expand()))
den = err._mathematica_().Denominator()._sage_().expand()
den = SR(str(MMA_simplify( den,full=False )).replace('Sqrt','sqrt')).simplify()
show(symround(den,tol=1e-30))

1

# Testing for auto

In [3]:
def obtain_exponent(expr):
    expr = SR(str(( expr)).replace('Sqrt','sqrt'))
    try:
        exp = expr.op[0].op[0].op[0].op[1]
    except:
        try:
            exp = expr.op[0].op[0].op[1]
        except:
            try:
                exp = expr.op[0].op[1]
            except:
                exp = 0
    return exp

def obtain_expr_exp(expr):
    try:
        el = 0    
        q = (expr._mathematica_().Numerator()._sage_() .expand().operands()[el])
        q = SR(str(( q)).replace('Sqrt','sqrt'))
        test_bool = (str(q._mathematica_().Numerator()._sage_().expand().denominator()) == str(1))
        num_exp = (obtain_exponent(q._mathematica_().Numerator()._sage_().expand()))
        den1_exp = (obtain_exponent(q._mathematica_().Denominator()._sage_().expand()))
        den2_exp = (obtain_exponent(expr._mathematica_().Denominator()._sage_().expand()))
        res = num_exp-den1_exp-den2_exp
        if not test_bool:
            #show(q)
            res = obtain_exponent(q.numerator())-obtain_exponent(q.denominator())
            return res
        return res
    except:
        #print('altern')
        den_exp = obtain_exponent( expr._mathematica_().Denominator()._sage_().expand())
        num_exp = obtain_exponent( expr._mathematica_().Numerator()._sage_().expand())
        return num_exp-den_exp


In [9]:
mx = -100
m = 1
loop1 = True
lindop = 0
while(loop1):
    try:
        show(lindop,eff_lind_coeff[lindop]  )
        pos = 0
        loop2 = True
        while(loop2):
            try: 
                err = eff_lind[lindop][pos].subs(Omega=1,kappa_c=5.61,kappa_b=1.79, R_f=2.1,Deg=np.sqrt(2)*C^m,Deg2=np.e*C^m,DEg=np.sqrt(5)*C^m,r_g=np.sqrt(3),gamma=0.3,r_f=0.1*np.sqrt(2), phi=1/10*np.pi,gamma_f=1.45,gamma_g=3)
                err = err._mathematica_().Together()._sage_() 
                exp = obtain_expr_exp(err)
                if exp > mx:
                    mx = exp
                pos += 1
                print(exp)
            except IndexError:
                loop2 = False
        lindop +=1
    except IndexError:
        loop1 = False

0 sqrt(kappa_c)

-2
-3/2
-3/2
-3/2


1 sqrt(gamma_g)

-2
-1
-1
-1


2 sqrt(gamma_f)

...

-3/2
-3/2


4 sqrt(gamma)

-3
-2


5 sqrt(kappa_b)

-2
-1
-1
-1


In [63]:
show(mx)

1.00000000000000

In [179]:
show(expr)

2.36854385646540*((2.2737367544323206e-13 - 5513.660907473835*I)*C^(7/2) + (2.842170943040401e-14 - 689.2076134342294*I)*C^(5/2))/(9426.862830620365*C^5 + (27285.3681027740 - 44079.8476731565*I)*C^4 + (8526.67753211688 - 98508.8580175064*I)*C^3 + (852.667753211688 - 29482.7478931105*I)*C^2 + (26.6458672878653 - 2878.20639191472*I)*C - 88.3799230197386*I)^1.00000000000000

7/2

In [188]:
#TEST
def obtain_exponent(expr):
    try:
        orde = SR(str(( expr)).replace('Sqrt','sqrt')).Order()
        exp = orde.op[0].op[0].op[0].op[1]
    except TypeError:
        exp = 0
    return exp

orde.op[0].op[0].op[0].op[1]

In [ ]:
el = 0
q = (symround(err._mathematica_().Numerator()._sage_() ,tol=1e-30 ).expand().operands()[el])

print('Numerator')

show(symround(q._mathematica_().Numerator()._sage_(),digits=3).expand())

print(obtain_exponent(symround(q._mathematica_().Numerator()._sage_(),digits=3).expand()))

print('\n\nDenominator')
show(symround(q._mathematica_().Denominator()._sage_(),digits=3).expand())
print(obtain_exponent(symround(q._mathematica_().Denominator()._sage_(),digits=3).expand()))

print('\n\nextra Denominator')
show( symround(err._mathematica_().Denominator()._sage_().expand(),digits=3))

print(obtain_exponent(symround(err._mathematica_().Denominator()._sage_().expand(),digits=3)))

In [36]:
lindop = 0
pos = 0 
show(symround(eff_lind[lindop][pos]))

-(-I*sqrt(C*gamma*kappa_c)*C^3*sqrt(R_f)*kappa_b^4*kappa_c^3*e^(2*I*phi) - I*sqrt(C*gamma*kappa_c)*C^3*sqrt(R_f)*kappa_b^4*kappa_c^3*e^(-2*I*phi) - 0.12*I*sqrt(C*gamma*kappa_c)*C^2*sqrt(R_f)*kappa_b^4*kappa_c^3*e^(2*I*phi) - 0.12*I*sqrt(C*gamma*kappa_c)*C^2*sqrt(R_f)*kappa_b^4*kappa_c^3*e^(-2*I*phi))*Omega*sqrt(kappa_c)/(C^4*DEg*gamma*kappa_b^4*kappa_c^4*e^(4*I*phi) + C^4*DEg*gamma*kappa_b^4*kappa_c^4*e^(-4*I*phi) - 2*C^4*DEg*gamma*kappa_b^4*kappa_c^4 + 2*I*C^4*R_f*gamma*kappa_b^4*kappa_c^4 - 0.50*I*C^4*gamma_f*kappa_b^4*kappa_c^4*e^(4*I*phi) - 0.50*I*C^4*gamma_g*kappa_b^4*kappa_c^4*e^(4*I*phi) - 0.50*I*C^4*gamma_f*kappa_b^4*kappa_c^4*e^(-4*I*phi) - 0.50*I*C^4*gamma_g*kappa_b^4*kappa_c^4*e^(-4*I*phi) - 4*C^3*DEg*gamma*kappa_b^4*kappa_c^4 + 1.2*I*C^3*R_f*gamma*kappa_b^4*kappa_c^4 + I*C^4*gamma_f*kappa_b^4*kappa_c^4 + I*C^4*gamma_g*kappa_b^4*kappa_c^4 - 1.2*C^2*DEg*gamma*kappa_b^4*kappa_c^4 + 0.19*I*C^2*R_f*gamma*kappa_b^4*kappa_c^4 + 2*I*C^3*gamma_f*kappa_b^4*kappa_c^4 + 2*I*C^3*gamma_g*kappa_b^4*kappa_c^4 - 0.12*C*DEg*gamma*kappa_b^4*kappa_c^4 + 0.0078*I*C*R_f*gamma*kappa_b^4*kappa_c^4 + 0.62*I*C^2*gamma_f*kappa_b^4*kappa_c^4 + 0.62*I*C^2*gamma_g*kappa_b^4*kappa_c^4 - 0.0039*DEg*gamma*kappa_b^4*kappa_c^4 + 0.062*I*C*gamma_f*kappa_b^4*kappa_c^4 + 0.062*I*C*gamma_g*kappa_b^4*kappa_c^4 + 0.0020*I*gamma_f*kappa_b^4*kappa_c^4 + 0.0020*I*gamma_g*kappa_b^4*kappa_c^4)

0.50*(C - 0.50*I*Deg + 0.25)*Omega^2/(C*(Deg + 0.50*I)*R_f*gamma + (C - 0.50*I*Deg + 0.25)*DEg*gamma + (0.50*I*C + 0.25*Deg + 0.12*I)*gamma_f + (0.50*I*C + 0.25*Deg + 0.12*I)*gamma_g)

In [7]:
show(symround(eff_hamiltonian_tayl[0,0]  ))

0.50*Omega^2*(c + 0.25)/((-0.50*I*C*R_f + DEg*c + 0.25*DEg)*gamma + (-0.50*I*c - 0.12*I)*gamma_f + (-0.50*I*c - 0.12*I)*gamma_g)

In [8]:
show(symround(eff_hamiltonian_tayl[1,1]  ))

0.50*((Deg - 0.50*I)*c - 0.50*I*C + 0.25*Deg - 0.12*I)*Omega^2/((C*(-0.50*I*Deg - 0.25)*R_f + (Deg*c - 0.50*I*C + 0.25*Deg - 0.50*I*c - 0.12*I)*DEg)*gamma + ((-0.50*I*Deg - 0.25)*c - 0.25*C - 0.12*I*Deg - 0.062)*gamma_f + ((-0.50*I*Deg - 0.25)*c - 0.25*C - 0.12*I*Deg - 0.062)*gamma_g)